In [2]:
import os
import re
import csv
import datetime
import firebirdsql

from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

True

In [3]:


def get_firebird_connection():
    # Ajustar com os parâmetros corretos do Firebird, inclusive charset
    return firebirdsql.connect(
        host=os.getenv('HOST'),
        port=int(os.getenv('PORT', '3050')),
        database=os.getenv('DB_PATH'),
        user=os.getenv('APP_USER'),
        password=os.getenv('PASSWORD'),
        role=os.getenv('ROLE'),
        auth_plugin_name=os.getenv('AUTH'),
        wire_crypt=False,
        charset='ISO8859_1'
    )

In [ ]:
conn = get_firebird_connection()
curs = conn.cursor()

# Pegar todos os dados das colunas HISTORICOPRODUTO (de 1 a 10), onde TIPO = 'PEDIDO'
for i in range(1, 11):
    curs.execute(f"""
        SELECT NUMDOCUMENTO, DATA
        FROM HISTORICOPRODUTO{i}
        WHERE TIPO = 'PEDIDO'
        AND DATA BETWEEN '2024-11-01' AND '2024-12-18'
        AND CDPRODUTO = 51005
    """)
    rows = curs.fetchall()
    for row in rows:
        print(row)

In [ ]:
import pandas as pd
import csv

# Caminho para o arquivo original
file_path = './arquivos/faturamento-comagro.csv'

# Ler o arquivo CSV com separador ';' e sem cabeçalho
data = pd.read_csv(file_path, sep=';', header=None, dtype=str, keep_default_na=False)

# Função para formatar datas de AAAA-MM-DD para DD/MM/AAAA
def format_date(date_str):
    try:
        # Tenta converter AAAA-MM-DD -> DD/MM/AAAA
        return pd.to_datetime(date_str, format='%Y-%m-%d').strftime('%d/%m/%Y')
    except ValueError:
        # Se falhar, devolve o original, sem mexer
        return date_str

# Aplicar a formatação de datas apenas na 4ª coluna (índice 3)
data[3] = data[3].apply(format_date)
print(data[3])

# Salvar o arquivo transformado
output_path = './arquivos/faturamento-comagro-transformado.csv'
data.to_csv(
    output_path,
    sep=';',           # mantém o separador ponto-e-vírgula
    index=False,       # não salva índice
    header=False,      # não salva cabeçalho
    quoting=csv.QUOTE_NONE,  # não coloca aspas
    escapechar='\\',   # se tiver ponto-e-vírgula ou barra
    encoding='utf-8'
)

print(f"Arquivo salvo em: {output_path}")

In [ ]:
import pandas as pd
import csv

# Caminho para o arquivo original
file_path = './arquivos/faturamento-comagro-cnpj-data-corrigido.csv'

# Ler o arquivo CSV com separador ';' e sem cabeçalho
data = pd.read_csv(file_path, sep=';', header=None, dtype=str, keep_default_na=False)

colunas_alvo = [17, 18, 22, 23, 24]

def trocar_ponto_por_virgula(valor_str):
    """
    Substitui todos os '.' por ',' no texto.
    Se por acaso quiser filtrar só se for número, poderia
    usar uma checagem adicional. Mas aqui, troca direta.
    """
    # valor_str é sempre string (dtype=str)
    return valor_str.replace('.', ',')

# Aplicar a função somente nas colunas-alvo
for col in colunas_alvo:
    data[col] = data[col].apply(trocar_ponto_por_virgula)

# Salvar o arquivo transformado
output_path = './arquivos/faturamento-comagro-transformado.csv'
data.to_csv(
    output_path,
    sep=';',           # mantém o separador ponto-e-vírgula
    index=False,       # não salva índice
    header=False,      # não salva cabeçalho
    quoting=csv.QUOTE_NONE,  # não coloca aspas
    escapechar='\\',   # se tiver ponto-e-vírgula ou barra
    encoding='utf-8'
)

print(f"Arquivo salvo em: {output_path}")

In [1]:

def normalizar_numero(valor):
    """
    Substitui o ponto decimal por vírgula, 
    arredonda para duas casas decimais se for numérico.
    Se 'valor' não for conversível em float, 
    apenas faz 'replace' de '.' por ','.
    """
    if not valor:
        return ""

    try:
        # Tenta converter em float
        f_val = float(valor)
        # Arredonda para duas casas decimais
        f_val = round(f_val, 2)
        # Formata com 2 casas decimais e troca '.' por ','
        val_str = f"{f_val:.2f}".replace('.', ',')
        return val_str
    except ValueError:
        # Se não der para converter em float,
        # apenas troca '.' por ',' no texto original
        val_str = str(valor).replace('.', ',')
        return val_str


In [ ]:
print(normalizar_numero(123.456789))  # Saída: "123,46"
print(normalizar_numero(123.4))       # Saída: "123,40"
print(normalizar_numero("123.456"))   # Saída: "123,46"
print(normalizar_numero("123.454"))   # Saída: "123,46"
print(normalizar_numero("abc"))       # Saída: ""
print(normalizar_numero(None))        # Saída: ""

In [6]:
import pysftp

load_dotenv()

cnopts = pysftp.CnOpts()

# Carrega o arquivo 'my_known_hosts' (ou outro nome)
cnopts.hostkeys.load('my_known_hosts')

# start_date = '2025-01-18'
start_ = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
end_date = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

date_start_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
date_end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")

csv_path = f"./arquivos/faturamento-comagro-{date_start_date.day}_{
        date_start_date.month}-{date_end_date.day}_{date_end_date.month}.csv"

# Função fictícia de SFTP (dados incompletos)
def enviar_arquivo_sftp(file_path):
    """
    Função para ilustrar uma conexão SFTP.
    Dados de host, user, password etc. ainda não estão definidos.
    """

    # Configuração do servidor SFTP
    sftp_host = os.getenv('SFTP_HOST')
    sftp_user = os.getenv('SFTP_USER')
    sftp_pass = os.getenv('SFTP_PASSWORD')
    remote_dir = '/workarea'

    with pysftp.Connection(host=sftp_host, username=sftp_user, password=sftp_pass, cnopts=cnopts) as sftp:
        with sftp.cd(remote_dir):
            sftp.put(file_path)  # envia o arquivo
            print("Arquivo enviado com sucesso.")

In [7]:
enviar_arquivo_sftp(csv_path)

Arquivo enviado com sucesso.


In [4]:
conn = get_firebird_connection()
cur = conn.cursor()

# Buscar todos os clientes com CPF_CNPJ igual a '000.000.000-00' ou '00.000.000/0000-00'
cur.execute("""
    SELECT CPF_CNPJ
    FROM CLIENTE
    WHERE NOME = 'CONSUMIDOR - A VISTA'
""")
clientes = cur.fetchall()

# Imprimir os nomes dos clientes
for cliente in clientes:
    print(cliente[0])

conn.close()

537.330.465-44
